In [72]:
import requests
import time
import uuid
import json

In [59]:
def parse_from_url(url: str):
    coords = []
    url = url.rsplit('/', maxsplit=1)[1]
    for coord in url.split(','):
        x, y = coord.split('%20')
        y = float(y)
        x = float(x)
        coords.append((x, y))
    return coords

def find_deltas(url: str):
    coords = parse_from_url(url)
    delta_x = max([x[0] for x in coords]) - min([x[0] for x in coords])
    delta_y = max([y[1] for y in coords]) - min([y[1] for y in coords])
    return delta_x, delta_y

def build_url(q4, q1, q2, q3):
    base = 'https://www.mlslistings.com/api/geo/intersect/'
    base += f"{q4[1]}%20{q4[0]},"
    base += f"{q1[1]}%20{q1[0]},"
    base += f"{q2[1]}%20{q2[0]},"
    base += f"{q3[1]}%20{q3[0]},"
    base += f"{q4[1]}%20{q4[0]}"
    return base


In [45]:
coords = parse_from_url('https://www.mlslistings.com/api/geo/intersect/-121.73914685991129%2037.464213992141545,-122.02891126420816%2037.45794607442863,-122.02650800493082%2037.182732129796086,-121.7398335054191%2037.18519385419808,-121.73914685991129%2037.464213992141545')
print(coords)
xmin = min([x[0] for x in coords])
xmax = max([x[0] for x in coords])
ymin = min([y[1] for y in coords])
ymax = max([y[1] for y in coords])

delta_x, delta_y = find_deltas('https://www.mlslistings.com/api/geo/intersect/-121.91185603598883%2037.2884878579338,-121.863619189065%2037.289034147836475,-121.86576495627692%2037.32740108598408,-121.91099772910407%2037.32801534386237,-121.91185603598883%2037.2884878579338')

[(-121.73914685991129, 37.464213992141545), (-122.02891126420816, 37.45794607442863), (-122.02650800493082, 37.182732129796086), (-121.7398335054191, 37.18519385419808), (-121.73914685991129, 37.464213992141545)]


In [27]:
def x_gen(xmin, xmax, delta_x):
    start = xmin
    while start < xmax:
        yield start
        start += delta_x
    yield xmax

for x in x_gen(xmin, xmax, delta_x):
    for y in x_gen(ymin, ymax, delta_y):
        print(f"{y} {x}")

37.182732129796086 -122.02891126420816
37.22738836660697 -122.02891126420816
37.27204460341785 -122.02891126420816
37.31670084022873 -122.02891126420816
37.36135707703961 -122.02891126420816
37.40601331385049 -122.02891126420816
37.45066955066137 -122.02891126420816
37.464213992141545 -122.02891126420816
37.182732129796086 -121.96591153886637
37.22738836660697 -121.96591153886637
37.27204460341785 -121.96591153886637
37.31670084022873 -121.96591153886637
37.36135707703961 -121.96591153886637
37.40601331385049 -121.96591153886637
37.45066955066137 -121.96591153886637
37.464213992141545 -121.96591153886637
37.182732129796086 -121.90291181352457
37.22738836660697 -121.90291181352457
37.27204460341785 -121.90291181352457
37.31670084022873 -121.90291181352457
37.36135707703961 -121.90291181352457
37.40601331385049 -121.90291181352457
37.45066955066137 -121.90291181352457
37.464213992141545 -121.90291181352457
37.182732129796086 -121.83991208818277
37.22738836660697 -121.83991208818277
37.27

In [60]:
coord_pairs = []
y = ymin
while y < ymax:
    x = xmax
    #print(f"{x} -> {x+delta_x} -> {xmax}")
    #input('press enter')
    while x > xmin:
        q1 = (y, x-delta_x)
        q2 = (y+delta_y, x-delta_x)
        q3 = (y+delta_y, x)
        q4 = (y, x)
        coord_pairs.append((q4, q1, q2, q3))
        x -= delta_x
    y += delta_y

coord_pairs

[((37.182732129796086, -121.73914685991129),
  (37.182732129796086, -121.78738370683512),
  (37.22225961572465, -121.78738370683512),
  (37.22225961572465, -121.73914685991129)),
 ((37.182732129796086, -121.78738370683512),
  (37.182732129796086, -121.83562055375894),
  (37.22225961572465, -121.83562055375894),
  (37.22225961572465, -121.78738370683512)),
 ((37.182732129796086, -121.83562055375894),
  (37.182732129796086, -121.88385740068277),
  (37.22225961572465, -121.88385740068277),
  (37.22225961572465, -121.83562055375894)),
 ((37.182732129796086, -121.88385740068277),
  (37.182732129796086, -121.9320942476066),
  (37.22225961572465, -121.9320942476066),
  (37.22225961572465, -121.88385740068277)),
 ((37.182732129796086, -121.9320942476066),
  (37.182732129796086, -121.98033109453043),
  (37.22225961572465, -121.98033109453043),
  (37.22225961572465, -121.9320942476066)),
 ((37.182732129796086, -121.98033109453043),
  (37.182732129796086, -122.02856794145426),
  (37.2222596157246

In [73]:
urls = []
data = {}
for i in coord_pairs:
    urls.append(build_url(*i))

In [77]:
for url in urls:
    if url in data:
        continue
    try:
        print(url)
        r = requests.get(url)
        print(r)
        response_data = r.json()
        response_data['url'] = url
        with open(f'{uuid.uuid4()}.json', 'w') as fp:
            json.dump(response_data, fp)
        data[url] = r.json()
        time.sleep(30)
    except KeyboardInterrupt:
        break

https://www.mlslistings.com/api/geo/intersect/-121.9320942476066%2037.38036955943892,-121.98033109453043%2037.38036955943892,-121.98033109453043%2037.419897045367485,-121.9320942476066%2037.419897045367485,-121.9320942476066%2037.38036955943892
<Response [200]>
https://www.mlslistings.com/api/geo/intersect/-121.98033109453043%2037.38036955943892,-122.02856794145426%2037.38036955943892,-122.02856794145426%2037.419897045367485,-121.98033109453043%2037.419897045367485,-121.98033109453043%2037.38036955943892
<Response [200]>
https://www.mlslistings.com/api/geo/intersect/-122.02856794145426%2037.38036955943892,-122.07680478837808%2037.38036955943892,-122.07680478837808%2037.419897045367485,-122.02856794145426%2037.419897045367485,-122.02856794145426%2037.38036955943892
<Response [200]>
https://www.mlslistings.com/api/geo/intersect/-121.73914685991129%2037.419897045367485,-121.78738370683512%2037.419897045367485,-121.78738370683512%2037.45942453129605,-121.73914685991129%2037.45942453129605,

In [79]:
print(len(data), len(coord_pairs))

56 56
